In [1]:
# tagtog에서 폴더 다운받기
import os
import requests

#로그인 위치
url = "https://tagtog.net/-login"

#다운로드 위치
file_url = 'https://www.tagtog.net/Jjackson/NLP-06-RE/-downloads/dataset-as-anndoc'
o_file = 'abc.zip'  
if os.path.exists(o_file):
    os.remove(o_file)

#로그인 정보
login_info = {
    'loginid' : "ID",
    'password' : "PW",
}
#로그인
with requests.Session() as s:
    login_req = s.post(url, data=login_info)
    r = s.get(file_url)

    with open(o_file,"wb") as output:
        output.write(r.content)

In [2]:
# tagtog 출력 파일 읽기
import json
import glob
import re
import os
import pandas as pd

In [3]:
# google sheet와 연결
# 참고 자료 : https://pbj0812.tistory.com/184

!pip install gspread
!pip install oauth2client

import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

#json key file 위치
json_file_name = './excellent-grin-332412-06e60916152b.json'

# json key file을 이용하여 접속
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_file_name, scope)
gc = gspread.authorize(credentials)

#구글 스프레드 시트 주소
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1IkQASpJTZjORCXgUrNhG47D0nb8Au_1fsNEnuUkMLBU/edit"

# 스프레드시트 문서 가져오기
doc = gc.open_by_url(spreadsheet_url)


#시트 선택하기
sheet_name = "total"
worksheet = doc.worksheet(sheet_name)
## 403 error가 뜰 경우, google sheets API를 활성시켜줘야 함


In [4]:

# # 시트의 모든 데이터 가져오기 #is not true. Shoud be: 시트의 모든 데이터를 가지고 와서 덮어써서 업로드하기
values = worksheet.get_all_values()
columns = values[0]
"""
['0',
 'baek_357',
 '인물:개발기술',
 '서비스 : 출시 주체',
 '인물 : 개발_기술',
 '인물 : 개발_기술',
 '인물 : 개발_기술',
 '인물 : 개발_기술',
 '인물 : 개발_기술',
 'ERROR',
 '',
 '<sbj:조셉 웨이젠바움>(Joseph Weizenbaum)의 <obj:ELIZA>는 사람들이 그들이 대화를 나누는 때때로 상대가 컴퓨터가 아니라 사람이라고 생각할 정도의 질로 대화했다.']
"""
from collections import Counter

rows = []
for i in range(1,len(values[1:])):
    
    annotations = values[i][2:-1]
    annotations = list(filter(None, annotations)) # remote empty string
    annotations = list(map(lambda x:x.replace("_", "").replace(" ", ""), annotations)) # convert some examples: e.g. ground truth: 개발기술 <-> annotation: 개발_기술 혹은 개발 기술
    counters = dict( Counter(annotations) )

    counters = dict(sorted(counters.items(), key = lambda x : x[1], reverse = True))

    # percentage of error
    err_count = 0
    total_count = 0
    for k, v in counters.items():
        if k == "ERROR":
            err_count += 1
        total_count += v

    # distribution of annotation
    top2_ann = list(counters.keys())[:2]
    first_ann = top2_ann[0]
    try:
        second_ann = top2_ann[1]
    except:
        # 만장일치! 이야호!
        rows.append( [ "만장일치! 이야호!",counters, values[i][-1] ] )
        continue

    if counters[first_ann] / total_count > 0.6:
        rows.append( ["과반 이상의 의견!", counters, values[i][-1]])
    else:
        rows.append(["다양한 의견!", counters, values[i][-1]])


    

In [5]:
df = pd.DataFrame(rows, columns = ["합치 여부", "counters", "문장"])
df


,합치 여부,counters,문장
0,과반 이상의 의견!,"{'인물:개발기술': 5, '서비스:출시주체': 1, 'ERROR': 1}",<sbj:조셉 웨이젠바움>(Joseph Weizenbaum)의 <obj:ELIZA>...
1,다양한 의견!,"{'인물:출판물': 4, 'ERROR': 2, '인물:개발기술': 1}",그 프로그램은 결국 러셀과 <sbj:화이트헤드>의 '<obj:수학 원리>'에 나오는...
2,다양한 의견!,"{'인물:출판물': 4, 'ERROR': 2, '인물:개발기술': 1}",그 프로그램은 결국 <sbj:러셀>과 화이트헤드의 '<obj:수학 원리>'에 나오는...
3,과반 이상의 의견!,"{'인물:개발기술': 6, '서비스:출시주체': 1}",<sbj:월터>의 <obj:거북이 로봇>이 이 아이디어를 중요하게 포함한 연구의 예이다.
4,다양한 의견!,"{'인물:개발기술': 3, 'ERROR': 2, '인물:출판물': 2}","다른 ""검색"" 프로그램은 기하학과 대수학의 문제를 해결하는 것처럼 인상적인 작업 -..."
...,...,...,...
390,만장일치! 이야호!,{'서비스:출시주체': 4},<obj:신한금융그룹>은 최근 왓슨을 도입하기 위한 태스크포스(TF)인 '보물섬 프...
391,만장일치! 이야호!,{'인물:출판물': 4},"<sbj:Haikonen>, Pentti (2003), <obj:The Cognit..."
392,만장일치! 이야호!,{'기술:정의': 4},<sbj:인공지능>(人工智能) 또는 아티피시알 인텔리전스(영어: Artificial...
393,만장일치! 이야호!,{'인물:출판물': 4},"<sbj:McCarthy, John> (1971-1987), <obj:General..."


In [6]:
df['label'] = df['counters'].map(lambda x:list(x.keys())[0])
label_count = df['label'].value_counts().sort_index()
label_count

ERROR       38
기술:개발단체     12
기술:개발일      21
기술:정의       58
기술:하위기술     90
서비스:기반기술    35
서비스:출시일     21
서비스:출시주체    33
인물:개발기술     43
인물:출판물      44
Name: label, dtype: int64

## 진행 방식
1. 각 문장 마다 가장 높은 순위로 나온 레이블을 일단 기준으로 둠
2. 각 문장에 레이블이 만장일치로 통일되면 합치여부를 만장일치로 분류
3. 각 문장에 가장 높은 순위로 나온 레이블의 확률이 60% 이상이면 합치여부를 과반 이상으로 분류함. 60%으로 설정한 이유는 전체 최대 7표 중에서(마스킹이 있으므로 7표가 미달될 때도 있음) 가장 많은 득표에 해당하는게 60%을 넘을 경우 분산되어도 가장 높은 득표의 레이블과 차이가 꽤 났기 때문
4. 3에 해당하지 않은 레이블에 대한 표수가 분산됨. 그래서 직접 눈으로 확인해볼 필요가 있을 것 같아서 합치여부를 다양한 의견으로 두었음!

# 각 레이블 별 7명 만장일치로 통일된 비율


In [7]:
# 잘 맞는 것
perfect = df[df['합치 여부'] == "만장일치! 이야호!"]
perfect['perfect_label'] = perfect['counters'].map(lambda x:list(x.keys())[0])
perfect_count = perfect_ratio = perfect['perfect_label'].value_counts().sort_index()
perfect_count, perfect_count / label_count, perfect

<ipython-input-7-23cb9bd7b0ba>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  perfect['perfect_label'] = perfect['counters'].map(lambda x:list(x.keys())[0])


(기술:개발단체      4
 기술:개발일       6
 기술:정의       41
 기술:하위기술     28
 서비스:기반기술     5
 서비스:출시일      6
 서비스:출시주체    11
 인물:개발기술     10
 인물:출판물      38
 Name: perfect_label, dtype: int64,
 ERROR            NaN
 기술:개발단체     0.333333
 기술:개발일      0.285714
 기술:정의       0.706897
 기술:하위기술     0.311111
 서비스:기반기술    0.142857
 서비스:출시일     0.285714
 서비스:출시주체    0.333333
 인물:개발기술     0.232558
 인물:출판물      0.863636
 dtype: float64,
           합치 여부         counters  \
 9    만장일치! 이야호!    {'인물:출판물': 7}   
 10   만장일치! 이야호!    {'인물:출판물': 7}   
 21   만장일치! 이야호!    {'기술:개발일': 7}   
 25   만장일치! 이야호!   {'인물:개발기술': 7}   
 32   만장일치! 이야호!    {'기술:개발일': 7}   
 ..          ...              ...   
 390  만장일치! 이야호!  {'서비스:출시주체': 4}   
 391  만장일치! 이야호!    {'인물:출판물': 4}   
 392  만장일치! 이야호!     {'기술:정의': 4}   
 393  만장일치! 이야호!    {'인물:출판물': 4}   
 394  만장일치! 이야호!    {'인물:출판물': 4}   
 
                                                     문장     label perfect_label  
 9    그러나 이에 반하는 손쉬운 예를 <sbj:사이먼 블랙번>의 철학 입문서 "<obj:...

# 만장일치이거나 가장 많은 선택을 받은 레이블이 60% 이상인 경우까지 포함한 경우 각 레이블 별 비율

In [8]:
most = df[df['합치 여부'] == "과반 이상의 의견!"]
most['most_label'] = most['counters'].map(lambda x:list(x.keys())[0])
most_count = most['most_label'].value_counts().sort_index()
( most_count + perfect_count ) / label_count

<ipython-input-8-df3f5b33b8ce>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  most['most_label'] = most['counters'].map(lambda x:list(x.keys())[0])


ERROR            NaN
기술:개발단체     0.666667
기술:개발일      0.761905
기술:정의       0.896552
기술:하위기술     0.666667
서비스:기반기술    0.742857
서비스:출시일     0.809524
서비스:출시주체    0.787879
인물:개발기술     0.813953
인물:출판물      0.954545
dtype: float64

In [11]:
# 한합쳐지는 경우
many = df[df['합치 여부'] == "다양한 의견!"]
many['many_label'] = many['counters'].map(lambda x:list(x.keys())[0])
many_count = many['many_label'].value_counts().sort_index()
( many_count + most_count + perfect_count ) / label_count#, most_count, most

<ipython-input-11-2ac9d6533e0f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  many['many_label'] = many['counters'].map(lambda x:list(x.keys())[0])


ERROR       NaN
기술:개발단체     1.0
기술:개발일      1.0
기술:정의       1.0
기술:하위기술     1.0
서비스:기반기술    1.0
서비스:출시일     1.0
서비스:출시주체    1.0
인물:개발기술     1.0
인물:출판물      1.0
dtype: float64

In [13]:
# 우리가 가장 합치되지 않은 레이블:기술:개발단체, 기술:하위기술, 기술:개발일, 서비스:기반기술, 서비스:출시일,서비스:출시주체


scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

#json key file 위치
json_file_name = './excellent-grin-332412-06e60916152b.json'

# json key file을 이용하여 접속
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_file_name, scope)
gc = gspread.authorize(credentials)

#구글 스프레드 시트 주소
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1xDrRN0_IqfLKo2hdSY_UjN7AyTU65r2MIHqxajS1U9U/edit"

# 스프레드시트 문서 가져오기
doc = gc.open_by_url(spreadsheet_url)

sheet_name = "기술:개발단체"

#시트 선택하기
worksheet = doc.worksheet(sheet_name)
## 403 error가 뜰 경우, google sheets API를 활성시켜줘야 함



# # 시트의 모든 데이터 가져오기 #is not true. Shoud be: 시트의 모든 데이터를 가지고 와서 덮어써서 업로드하기
values = worksheet.get_all_values()


df[df['label'] == "기술:개발단체"].sort_values(by = ["합치 여부"]).to_csv("2.csv")
df[df['label'] == "기술:하위기술"].sort_values(by = ["합치 여부"]).to_csv("3.csv")
df[df['label'] == "기술:개발일"].sort_values(by = ["합치 여부"]).to_csv("4.csv")
df[df['label'] == "서비스:출시일"].sort_values(by = ["합치 여부"]).to_csv("5.csv")
df[df['label'] == "서비스:출시주체"].sort_values(by = ["합치 여부"]).to_csv("6.csv")
df[df['label'] == "서비스:출시주체"].sort_values(by = ["합치 여부"]).to_csv("7.csv")
df[df['label'] == "인물:개발기술"].sort_values(by = ["합치 여부"]).to_csv("1.csv")


# 기술 개발단체는 서비스 출시주체와 자주 갈리는 것처럼 기술과 서비스의 정의가 명확하지 않아서 갈리는 패턴이 존재

In [14]:
df[df['label'] == "기술:개발단체"].sort_values(by = ["합치 여부"])

,합치 여부,counters,문장,label
29,과반 이상의 의견!,"{'기술:개발단체': 5, 'ERROR': 2}","1951년에, <obj:맨체스터 대학>의 <sbj:페란티 마크 1>(Ferranti...",기술:개발단체
54,과반 이상의 의견!,"{'기술:개발단체': 6, '서비스:출시주체': 1}","다른 프로그램은 목표와 목표에 다가가기 위한 세부 계획을 검색했고, 여기에는 <ob...",기술:개발단체
253,과반 이상의 의견!,"{'기술:개발단체': 3, '서비스:출시주체': 1}",<obj:딥마인드>에서는 <sbj:화면의 정보만을 가지고 비디오 게임을 어떻게 플레...,기술:개발단체
325,과반 이상의 의견!,"{'기술:개발단체': 4, '서비스:출시주체': 1}",원래 시작은 미국 국방부의 <obj:DARPA>에서 진행한 일종의 인공지능프로그램개...,기술:개발단체
147,다양한 의견!,"{'기술:개발단체': 4, '서비스:출시주체': 2, 'ERROR': 1}",2012년 스탠포드대학의 앤드류 응과 <obj:구글>이 함께한 <sbj:딥 러닝 프...,기술:개발단체
279,다양한 의견!,"{'기술:개발단체': 3, '서비스:출시주체': 2}",<obj:마이크로소프트>도 SW개발환경인 <sbj:Robotic Studio>를 개...,기술:개발단체
335,다양한 의견!,"{'기술:개발단체': 3, 'ERROR': 2}","리스프 프로그래밍 언어를 설계 및 구현하였으며, 1956년에 <obj:다트머스 학회...",기술:개발단체
354,다양한 의견!,"{'기술:개발단체': 2, 'ERROR': 2}",<obj:딥마인드>(영어: DeepMind Technologies Limited)는...,기술:개발단체
161,만장일치! 이야호!,{'기술:개발단체': 7},이와 달리 2006년 <obj:구글>은 수억 건의 문장과 번역문을 데이터베이스화하여...,기술:개발단체
307,만장일치! 이야호!,{'기술:개발단체': 4},2009년부터 2012년동안 스위스 AI 연구실 IDSIA에서 <obj:위르겐 슈밋...,기술:개발단체


In [15]:
df[df['label'] == "기술:하위기술"].sort_values(by = ["합치 여부"])

,합치 여부,counters,문장,label
48,과반 이상의 의견!,"{'기술:하위기술': 5, 'ERROR': 1, '서비스:기반기술': 1}",<obj:의미 망>을 사용하는 <sbj:첫번째 AI 프로그램>은 로스 퀄리언(Ros...,기술:하위기술
190,과반 이상의 의견!,"{'기술:하위기술': 5, 'ERROR': 1}",클라우드 및 모바일 환경으로 접어들면서 물리/가상화 IT 시스템의 복잡성이 더욱 높...,기술:하위기술
205,과반 이상의 의견!,"{'기술:하위기술': 2, 'ERROR': 1}","<sbj:딥 러닝>의 부활 이후 다양한 분야, 특히 <obj:자동 음성 인식>(AS...",기술:하위기술
214,과반 이상의 의견!,"{'기술:하위기술': 2, 'ERROR': 1}",<obj:자연어 처리>와 같은 <sbj:인공지능> 기술에 대해서 잘 알지 못해도 적...,기술:하위기술
216,과반 이상의 의견!,"{'기술:하위기술': 2, '서비스:기반기술': 1}",지금까지의 연구는 주로 <obj:추출 기법>에 집중되어 있으며 이는 <sbj:이미지...,기술:하위기술
...,...,...,...,...
229,만장일치! 이야호!,{'기술:하위기술': 3},"데이터 마이닝이란 기존 데이터베이스 관리도구의 데이터 수집, 저장, 관리, 분석의 ...",기술:하위기술
159,만장일치! 이야호!,{'기술:하위기술': 7},<obj:자동 데이터 요약>은 기계 학습 및 <sbj:데이터 마이닝>의 한 부분이다.,기술:하위기술
158,만장일치! 이야호!,{'기술:하위기술': 7},<obj:자동 데이터 요약>은 <sbj:기계 학습> 및 데이터 마이닝의 한 부분이다.,기술:하위기술
115,만장일치! 이야호!,{'기술:하위기술': 7},"상기 특징을 가진 빅 데이터의 분석, 활용을 위한 <sbj:빅 데이터 처리 기법>은...",기술:하위기술
